# Tri de listes

## Principe

Les algorithmes vu au chapitre 3 ne sont pas directement applicables aux listes simplement ou doublement chainées

Il n'y a pas d'accès indexé aux éléments

Le fournir serait trop cher en terme de complexité

Mais en a-t-on vraiment besoin ? 

Cela dépend des algorithmes

## Tri à bulle et par sélection

Pour rappel, en voici les codes du chapitre 3 pour trier des tableaux

In [1]:
def tri_a_bulles(T):
    N = len(T)
    for k in range(N,1,-1):    
        for i in range(0,k-1):  
            if T[i] > T[i+1]: 
                T[i],T[i+1] = T[i+1],T[i]  

In [2]:
def tri_par_selection(T):
    N = len(T)
    for i in range(0,N-1):
        jMin = i
        for j in range(i+1,N):
            if T[j] < T[jMin]: jMin = j
        T[i],T[jMin] = T[jMin],T[i]  

Dans les deux cas, il y a 

* deux parcours croissants
* des échanges d'éléments 

Les parcours croissants peuvent aisément être effectués par les itérateurs sur listes simples ou doubles. Par exemple, pour le tri par sélection

In [3]:
def tri_par_selection(L):
    i = L.begin()
    while i != L.end():
        jMin = i.copie()
        j = i.suivant(); 
        while j != L.end():
            if j.get_val() < jMin.get_val():
                jMin = j.copie()
            j.incr()
        echanger(i,jMin)
        i.incr()

Avec une fonction d'échange standard entre les éléments itérés, on a

In [4]:
def echanger(i,j):
    tmp = i.get_val()
    i.set_val(j.get_val())
    j.set_val(tmp)

In [5]:
import include.liste_double as ld
T = [ 6, 3, 5, 4, 1, 2, 8, 7, 9 ]; 

L = ld.Liste()
for t in T: L.push_back(t)

tri_par_selection(L); print(L)

⌀ ← 1 ⇄ 2 ⇄ 3 ⇄ 4 ⇄ 5 ⇄ 6 ⇄ 7 ⇄ 8 ⇄ 9 → ⌀


Mais il y a une autre manière d'échanger les éléments sans devoir les copier: échanger la place de leurs maillons dans la chaine. 

In [6]:
def echanger_emplacement_des_maillons(m1,m2):
    if m1 == m2: return
    
    m1p = m1.precedent; m1s = m1.suivant
    m2p = m2.precedent; m2s = m2.suivant
    
    m1p.suivant, m2p.suivant = m2p.suivant, m1p.suivant
    m1.suivant, m2.suivant = m2.suivant, m1.suivant
    
    m1s.precedent, m2s.precedent = m2s.precedent, m1s.precedent
    m1.precedent, m2.precedent = m2.precedent, m1.precedent 
    
def echanger_iterateurs(i,j):
    if i == j: return 
    echanger_emplacement_des_maillons(i._list_iterator__M, 
                                      j._list_iterator__M)

Mais attention, après échange, les itérateurs pointent toujours sur les mêmes éléments, mais ne sont plus au même emplacement de la liste

In [7]:
L = ld.Liste()
for t in T: L.push_back(t)
i = L.begin().suivant(0)
j = L.begin().suivant(4)

print(i,j,L)

echanger_iterateurs(i,j)

print(i,j,L)

6 1 ⌀ ← 6 ⇄ 3 ⇄ 5 ⇄ 4 ⇄ 1 ⇄ 2 ⇄ 8 ⇄ 7 ⇄ 9 → ⌀
6 1 ⌀ ← 1 ⇄ 3 ⇄ 5 ⇄ 4 ⇄ 6 ⇄ 2 ⇄ 8 ⇄ 7 ⇄ 9 → ⌀


Après échange des maillons, c'est `jMin` qui stocke maintenant l'emplacement dans la boucle sur `i`

In [8]:
def tri_par_selection(L):
    i = L.begin()
    while i != L.end():
        jMin = i.copie()
        
        j = i.suivant(); 
        while j != L.end():
            if j.get_val() < jMin.get_val():
                jMin = j.copie()
            j.incr()
            
        echanger_iterateurs(i,jMin)
        i = jMin.suivant()    # et pas i.incr()  

In [9]:
import include.liste_double as ld
T = [ 6, 3, 5, 4, 1, 2, 8, 7, 9 ]; 
L = ld.Liste()
for t in T: L.push_back(t)
tri_par_selection(L); print(L)

⌀ ← 1 ⇄ 2 ⇄ 3 ⇄ 4 ⇄ 5 ⇄ 6 ⇄ 7 ⇄ 8 ⇄ 9 → ⌀


## Tri par insertion

Nous avons déjà vu la traduction du tri par insertion en itérateurs dans la section sur les listes

In [10]:
def tri_par_insertion(L):
    if L.size() < 2: return
    
    k = L.begin().suivant()
    while k != L.end():
        tmp = k.get_val()
        
        j = k; i = j.precedent()
        while j != L.begin() and tmp < i.get_val():
            j.set_val(i.get_val())
            j = i.copie() 
            i.decr()
            
        j.set_val(tmp)
        k.incr()

In [11]:
L = ld.Liste()
for t in T: L.push_back(t)
print(L)

⌀ ← 6 ⇄ 3 ⇄ 5 ⇄ 4 ⇄ 1 ⇄ 2 ⇄ 8 ⇄ 7 ⇄ 9 → ⌀


In [12]:
tri_par_insertion(L)
print(L)

⌀ ← 1 ⇄ 2 ⇄ 3 ⇄ 4 ⇄ 5 ⇄ 6 ⇄ 7 ⇄ 8 ⇄ 9 → ⌀


Ce code est une transcription exacte de celui sur les tableaux. Mais ce n'est pas efficace. 

Le but de la boucle interne est de 

* trouver la position `i` où il faut insérer l'élément `k`
* déplacer l'élément de `k` vers `i`

Dans un tableau, cela requiert de déplacer tous les éléments entre `i` et `k-1` de 1 vers la droite. Mais dans une liste chainée, cela n'a pas de sens. Il suffit de retirer le maillon `k` et de l'insérer tel quel devant `i`,

Ecrivons un telle fonction

In [13]:
def deplacer_maillon(FROM,TO):
    if FROM == TO: return
    
    FROM.precedent.suivant = FROM.suivant
    FROM.suivant.precedent = FROM.precedent
    
    TO.precedent.suivant = FROM
    FROM.precedent = TO.precedent
    
    TO.precedent = FROM
    FROM.suivant = TO 
    
def deplacer_iterateur(FROM,TO):
    deplacer_maillon(FROM._list_iterator__M,
                     TO._list_iterator__M)

Elle nous permet de ré-écrire le tri par insertion

In [14]:
def tri_par_insertion_sans_copie(L):
    if L.size() < 2: return
    
    k = L.begin().suivant()
    while k != L.end():    
        i = k.copie()
        while i != L.begin() and \
              k.get_val() < i.precedent().get_val():
            i.decr()
        n = k.suivant()
        deplacer_iterateur(k,i)
        k = n

In [15]:
T = [ 6, 3, 5, 4, 1, 2, 8, 7, 9 ]
L = ld.Liste()
for t in T: L.push_back(t)

tri_par_insertion_sans_copie(L)
print(L)

⌀ ← 1 ⇄ 2 ⇄ 3 ⇄ 4 ⇄ 5 ⇄ 6 ⇄ 7 ⇄ 8 ⇄ 9 → ⌀


<table style="width: 100%; border: 0px">
<tr style="background-color:white; border:0px">
<td style="width: 120px; border: 0px">
    <img src="https://heig-vd.ch/ResourcePackages/WhiteFox/assets/images/logo-heig-vd.svg" height=200px align=left >
    </td>
    <td style="vertical-align: middle; border: 0px" height=200px>
    <p style="text-align: left">
        <a href="https://ocuisenaire.github.io/ASD1-notebooks/">ASD1 Notebooks on GitHub.io</a>
 </p>        
<p style="text-align: left">
© Olivier Cuisenaire, 2018 </p>
</td>
</tr>
</table>